In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:

from sklearn.metrics import roc_curve
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix as cmt
from sklearn import datasets, metrics, model_selection, svm
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve

import matplotlib.pyplot as plt  # doctest: +SKIP



In [ ]:
results={}

In [ ]:
train='https://raw.githubusercontent.com/defcom17/NSL_KDD/master/KDDTrain%2B.csv'
test ='https://raw.githubusercontent.com/defcom17/NSL_KDD/master/KDDTest%2B.csv'

In [ ]:
import pandas as pd
train=pd.read_csv(train)
test=pd.read_csv(test)


In [ ]:
Columns = ['duration', 'protocol_type', 'service', 'flag', 'src_bytes',
'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot',
'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell',
'su_attempted', 'num_root', 'num_file_creations', 'num_shells',
'num_access_files', 'num_outbound_cmds', 'is_host_login',
'is_guest_login', 'count', 'srv_count', 'serror_rate',
'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate',
'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count',
'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate',
'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
'dst_host_srv_rerror_rate', 'attack_type', 'difficulty_level']



In [ ]:
train.columns=Columns
test.columns=Columns

In [ ]:
#df=pd.read_csv('../input/resultsnow/results.csv')
#df

In [ ]:
total=pd.concat([train,test])
total.shape

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test = train_test_split(
    total, test_size=0.35, random_state=42)

In [ ]:
x_label=x_train['attack_type']
y_label=x_test['attack_type']

In [ ]:
x_label.shape

In [ ]:
xt=x_label
xt=np.where(xt=='normal',0 ,xt)
xt=np.where(xt!=0,1 ,xt)
x_label=xt.astype(int)


y=y_label
y=np.where(y=='normal',0 ,y)
y=np.where(y!=0,1 ,y)
y_label=y.astype(int)

In [ ]:
x_label.shape

In [ ]:

x_train=x_train.drop(['attack_type','difficulty_level'],1)
x_test=x_test.drop(['attack_type','difficulty_level'],1)
print(x_train.shape,x_test.shape)

In [ ]:

newtrain=pd.get_dummies(x_train)

newtrain.shape


In [ ]:
newtest=pd.get_dummies(x_test)

newtest.shape

In [ ]:
missed_columns=set(newtrain.columns)-set(newtest.columns)
missed_columns

In [ ]:
for i in missed_columns:
    newtest[i]=0


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE


clf=RandomForestClassifier(n_estimators=100,random_state=42,n_jobs=2,max_depth=2)

rfe=RFE(estimator=clf,n_features_to_select=30,step=1)

rfe.fit(newtrain,x_label.astype(int))
x_train=rfe.transform(newtrain)

x_test=rfe.transform(newtest)

print(x_test.shape,x_train.shape)

In [ ]:

from sklearn.metrics import roc_curve
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix as cmt
from sklearn import datasets, metrics, model_selection, svm
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve

import matplotlib.pyplot as plt  # doctest: +SKIP


In [ ]:
scaler1 = preprocessing.StandardScaler().fit(x_train)
x_train=scaler1.transform(x_train) 

scaler1 = preprocessing.StandardScaler().fit(x_test)
x_test=scaler1.transform(x_test) 

In [ ]:
from sklearn.ensemble import RandomForestClassifier

import time


clf=RandomForestClassifier(n_estimators=1000,random_state=12,
                           n_jobs=100,max_depth=100,
                           max_features='auto')
clf.fit(x_train,x_label)


x=clf.score(x_test,y_label)
print(x)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

import time
st=time.time()
#train
#clf=RandomForestClassifier(n_estimators=100,random_state=42,n_jobs=10,max_depth=100)
clf.fit(x_train,x_label)
sp=time.time()
rclf_time=sp-st
print('time',rclf_time)
results['rtime']=rclf_time


#test
st=time.time()

x=clf.score(x_test,y_label)
sp=time.time()
rclf_test_time=sp-st
print('test_time',rclf_test_time)

results['r_test_time']=rclf_test_time

y_pred=clf.predict(x_test)


print('scre',x)
results['rscore']=x

porbs=clf.predict_proba(x_test)
from sklearn.metrics import roc_curve
tpr,fpr,tr=roc_curve(y_label,porbs[:,1])
pscre, rscre, thresholds = precision_recall_curve(y_label, porbs[:,1])
results['rtpr']=tpr
results['rfpr']=fpr

results['rpscre']=pscre
results['rrscre']=rscre

auc=np.trapz(fpr,tpr)
results['rrocscore']=auc
pauc=np.trapz(pscre,rscre)
pauc
results['rprcscore']=pauc

In [ ]:
cmt(y_label,y_pred)

In [ ]:
def metrics(y_label,y_pred):
    cm=cmt(y_label,y_pred)
    tp=cm[0,0]
    fp=cm[0,1]
    fn=cm[1,0]
    tn=cm[1,1]
    precision=tp/(tp+fp)
    mapscore=average_precision_score(y_label,y_pred)
    recall=tp/(tp+fn)
    F1 = 2 * (precision * recall) / (precision + recall)
    print('precsion:',precision,'recall:' ,recall ,'F-score :',F1,mapscore)
    results['rmap']=mapscore
    results['rprec']=precision
    results['rrec']=recall
    results['rf']=F1
    print('acuu:',(tp+tn)/(tp+tn+fn+fp))
    return precision,recall,F1,mapscore
    
metrics(y_label,y_pred)

In [ ]:
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler,MinMaxScaler

import time 

st=time.time()
model1=Pipeline([('scaler',StandardScaler()),('svc',SVC(gamma='auto',probability=True))])


model1.fit(x_train,x_label)
stp=time.time()
svclf_time=stp-st
svclf_time




In [ ]:
results['stime']=svclf_time

In [ ]:
st=time.time()
score=model1.score(x_test,y_label)
stp=time.time()
svclf_test_time=stp-st
svclf_test_time
results['s_test_time']=svclf_test_time
results['sscore']=score
pred=model1.predict(x_test)

In [ ]:
mpred=model1.predict_proba(x_test)
pred=model1.predict(x_test)
tpr,fpr,thg=roc_curve(y_label,mpred[:,1])
pscre, rscre, thresholds = precision_recall_curve(y_label,mpred[:,1])
results['stpr']=tpr
results['sfpr']=fpr

results['spscre']=pscre
results['srscre']=rscre

def smetrics(y_label,y_pred):
    cm=cmt(y_label,y_pred)
    tp=cm[0,0]
    fp=cm[0,1]
    fn=cm[1,0]
    tn=cm[1,1]
    precision=tp/(tp+fp)
    mapscore=average_precision_score(y_label,y_pred)
    recall=tp/(tp+fn)
    F1 = 2 * (precision * recall) / (precision + recall)
    print('precsion:',precision,'recall:' ,recall ,'F-score :',F1)
    results['smap']=mapscore
    results['sprec']=precision
    results['srec']=recall
    results['sf']=F1
    return precision,recall,F1,mapscore
    
smetrics(y_label,pred)

auc=np.trapz(fpr,tpr)
print(auc)
results['srocscore']=auc
pauc=np.trapz(pscre,rscre)
pauc
results['sprcscore']=pauc

## xgb

In [ ]:
import xgboost as xgb
xclf = xgb.XGBRegressor(objective ='reg:tweedie', colsample_bytree = 0.3, learning_rate = 0.01,
                max_depth = 5, alpha = 10, n_estimators = 1000)


In [ ]:
x_test.shape

In [ ]:

## training
st=time.time()
xclf.fit(x_train,x_label)
stp=time.time()
tot=stp-st
print(tot)
results['xtime']=tot

In [ ]:

#test

y_pred=xclf.predict(x_test) 
st=time.time()

x=xclf.score(x_test,y_label)
print('score',x)
sp=time.time()
rclf_test_time=sp-st
print('test_time',rclf_test_time)

results['x_test_time']=rclf_test_time

y_pred=xclf.predict(x_test) 


from sklearn.metrics import roc_curve

from sklearn.metrics import accuracy_score
tpr,fpr,tr=roc_curve(y_label,y_pred)
pscre, rscre, thresholds = precision_recall_curve(y_label, y_pred)
results['xtpr']=tpr
results['xfpr']=fpr

results['xpscre']=pscre
results['xrscre']=rscre

auc=np.trapz(fpr,tpr)
print(auc)
results['xrocscore']=auc
pauc=np.trapz(pscre,rscre)
pauc
results['xprcscore']=pauc

In [ ]:
y_pred=np.where(y_pred>0.5,1,y_pred)
y_pred=np.where(y_pred<=0.5,0,y_pred)

In [ ]:
def xmetrics(y_label,y_pred):
    cm=cmt(y_label,y_pred)
    tp=cm[0,0]
    fp=cm[0,1]
    fn=cm[1,0]
    tn=cm[1,1]
    precision=tp/(tp+fp)
    mapscore=average_precision_score(y_label,y_pred)
    recall=tp/(tp+fn)
    F1 = 2 * (precision * recall) / (precision + recall)
    print('precsion:',precision,'recall:' ,recall ,'F-score :',F1)
    results['xmap']=mapscore
    results['xprec']=precision
    results['xrec']=recall
    results['xf']=F1
    x=(tp+tn)/(tp+tn+fp+fn)
    results['xscore']=x
    return precision,recall,F1,mapscore
    
xmetrics(y_label,y_pred)

## LSTM

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional
from keras.optimizers import SGD
import math
from sklearn.metrics import mean_squared_error
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
x_test = np.reshape(x_test, (x_test.shape[0],x_test.shape[1],1))

x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))

In [ ]:
# The LSTM architecture
regressor = Sequential()
# First LSTM layer with Dropout regularisation
regressor.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1],1)))
regressor.add(Dropout(0.2))
# Second LSTM layer
regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(0.2))
# Third LSTM layer
regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(0.2))
# Fourth LSTM layer
regressor.add(LSTM(units=50))
regressor.add(Dropout(0.2))
# The output layer
regressor.add(Dense(units=1))

# Compiling the RNN
regressor.compile(optimizer='rmsprop',loss='mean_squared_error')
# Fitting to the training set
st=time.time()
regressor.fit(x_train,x_label,epochs=20,batch_size=32)
sp=time.time()
tot=sp-st

results['ltime']=tot


In [ ]:
regressor.save('lmodel1.model')

In [ ]:
import time 
st=time.time()

sd=regressor.predict(x_test)
sp=time.time()
tot=sp-st

sd.shape
results['l_test_time']=tot

In [ ]:
sd=np.array(sd)
y_pred=np.where(sd>0.5,1,sd)
y_pred=np.where(y_pred!=1,0,y_pred)
y_pred[:10]

In [ ]:
from sklearn.metrics import roc_curve

from sklearn.metrics import accuracy_score
tpr,fpr,tr=roc_curve(y_label,y_pred)
pscre, rscre, thresholds = precision_recall_curve(y_label, y_pred)
results['ltpr']=tpr
results['lfpr']=fpr

results['lpscre']=pscre
results['lrscre']=rscre

auc=np.trapz(fpr,tpr)
print(auc)
results['lrocscore']=auc
pauc=np.trapz(pscre,rscre)
pauc
results['lprcscore']=pauc

In [ ]:

def lmetrics(y_label,y_pred):
    cm=cmt(y_label,y_pred)
    tp=cm[0,0]
    fp=cm[0,1]
    fn=cm[1,0]
    tn=cm[1,1]
    precision=tp/(tp+fp)
    mapscore=average_precision_score(y_label,y_pred)
    recall=tp/(tp+fn)
    F1 = 2 * (precision * recall) / (precision + recall)
    print('precsion:',precision,'recall:' ,recall ,'F-score :',F1)
    results['lmap']=mapscore
    results['lprec']=precision
    results['lrec']=recall
    results['lf']=F1
    ascore =(tp+tn)/(tp+tn+fp+fn)
    results['lscore']=ascore
    print('acc',ascore,)
    return precision,recall,F1,mapscore
    
lmetrics(y_label.astype(int),y_pred)


## auto encoders

In [ ]:
tot=pd.concat([train ,test])
tot.shape

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test = train_test_split(
    tot, test_size=0.35, random_state=42)

In [ ]:
mask=x_train.drop('difficulty_level',1)
mask.shape

In [ ]:
mask.attack_type=np.where(mask.attack_type=='normal',0,mask.attack_type)
mask.attack_type=np.where(mask.attack_type!=0,1,mask.attack_type)

In [ ]:
x_normal=mask
x_normal.shape


In [ ]:
x_normal_train=x_normal.attack_type==0
x_train1=x_normal[x_normal_train]
x_train1.shape

In [ ]:
x_train=x_train1.drop('attack_type',1)
y_test=x_test['attack_type']
y_test=np.where(y_test=='normal',0,y_test)
y_test=np.where(y_test!=0,1,y_test)
x_test=x_test.drop(['attack_type','difficulty_level'],1)
print(x_train.shape,x_test.shape,y_test.shape)

In [ ]:
x_test.columns

In [ ]:
x_train=pd.get_dummies(x_train)
x_test=pd.get_dummies(x_test)
print(x_train.shape,x_test.shape,y_test.shape)

In [ ]:
cols=set(x_test.columns)-set(x_train.columns)
for i in cols:
    x_train[i]=0
print(x_train.shape,x_test.shape,y_test.shape)

In [ ]:
cols=set(x_train.columns)-set(x_test.columns)
for i in cols:
    x_test[i]=0
print(x_train.shape,x_test.shape,y_test.shape)

In [ ]:
x_normal1=x_normal.drop('attack_type',1)
x_normal1=pd.get_dummies(x_normal1)
x_normal1.shape

In [ ]:
x_label=x_normal.attack_type
x_label.shape

In [ ]:
cols=set(x_normal1.columns)-set(x_train.columns)
cols

In [ ]:
for i in cols:
    x_train[i]=0
x_train.shape

In [ ]:
cols=set(x_normal1.columns)-set(x_test.columns)
cols

In [ ]:
for i in cols:
    x_test[i]=0
x_test.shape

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE


clf=RandomForestClassifier(n_estimators=100,random_state=42,n_jobs=2,max_depth=2)

rfe=RFE(estimator=clf,n_features_to_select=30,step=1)

rfe.fit(x_normal1,x_label.astype(int))

x_train=rfe.transform(x_train)
x_train.shape
x_test=rfe.transform(x_test)
x_test.shape

In [ ]:
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
scaler1 = preprocessing.StandardScaler().fit(x_train)
x_train=scaler1.transform(x_train) 


scaler2 = preprocessing.StandardScaler().fit(x_test)
x_test=scaler2.transform(x_test)

In [ ]:

from sklearn import metrics
import numpy as np
import pandas as pd
from IPython.display import display, HTML 
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation

model = Sequential()
model.add(Dense(25, input_dim=x_train.shape[1], activation='relu'))
model.add(Dense(3, activation='relu')) # size to compress to
model.add(Dense(25, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(x_train.shape[1])) # Multiple output neurons
model.compile(loss='mean_squared_error', optimizer='adam')


In [ ]:
import time
st=time.time()
model.fit(x_train,x_train,verbose=1,epochs=40)
sp=time.time()
atime=sp-st
results['atime']=atime


In [ ]:
import time
st-time.time()
pred = model.predict(x_test)
pred.shape
sp=time.time()
results['a_test_time']=sp-st

In [ ]:
x=np.mean(np.power(pred-x_test,2),axis=1)
x.shape

In [ ]:
error_df = pd.DataFrame({'Reconstruction_error': x,
                        'True_class':y_test.astype(int) })
error_df.describe()

In [ ]:
fpr.shape

In [ ]:
fpr, tpr, thresholds = roc_curve(error_df.True_class, error_df.Reconstruction_error)

plt.plot(fpr,tpr)

In [ ]:
pscre, rscre, thresholds = precision_recall_curve(error_df.True_class, error_df.Reconstruction_error)
results['apscre']=pscre
results['arscre']=rscre

In [ ]:
rec_er=error_df.Reconstruction_error
rec_er=np.where(rec_er>=0.4,1,rec_er)
y_pred=np.where(rec_er!=1,0,rec_er)


In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test.astype(int),y_pred) 
cm

In [ ]:
from sklearn.metrics import (confusion_matrix, precision_recall_curve, auc,
                             roc_curve, recall_score, classification_report, f1_score,
                             precision_recall_fscore_support)
from ipykernel import kernelapp as app
roc_auc = auc(tpr, fpr)

def ametrics(y_label,y_pred):
    cm=cmt(y_label,y_pred)
    tp=cm[0,0]
    fp=cm[0,1]
    fn=cm[1,0]
    tn=cm[1,1]
    precision=tp/(tp+fp)
    mapscore=average_precision_score(y_label,y_pred)
    recall=tp/(tp+fn)
    F1 = 2 * (precision * recall) / (precision + recall)
    print('precsion:',precision,'recall:' ,recall ,'F-score :',F1)
    results['amap']=mapscore
    results['aprec']=precision
    results['arec']=recall
    results['af']=F1
    ascore =(tp+tn)/(tp+tn+fp+fn)
    results['ascore']=ascore
    print('acc',ascore,)
    return precision,recall,F1,mapscore
    
ametrics(y_test.astype(int),y_pred)





auc=np.trapz(tpr,fpr)
print(auc)
results['arocscore']=auc
pauc=np.trapz(pscre,rscre)
pauc
results['aprcscore']=abs(pauc)
print(pauc)
results['atpr']=tpr
results['afpr']=fpr

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# ACCURACY
fig,ax=plt.subplots(figsize=(15,10))
x=int(results['rscore']*100)
ax.text(0,2+x,x, fontsize=12)
y=int(abs(results['xscore']*100))
ax.text(1,2+y,y, fontsize=12)
z=int(results['sscore']*100)
ax.text(2,2+z,z, fontsize=12)
w=int(results['ascore']*100)
ax.text(3,2+w,w, fontsize=12)
v=int(results['lscore']*100)
ax.text(4,2+v,v, fontsize=12)

labels=['Random Forest','XGB','SVM','AUTO_ENOCDERS','LSTM']

ax.bar(labels,[x,y,z,w,v])
plt.title('ACCURACY COMPARISION COMBINED KDD(65:35)')

In [ ]:
np.trapz(results['atpr'],results['afpr'])

In [ ]:
import sys
fig,ax=plt.subplots(figsize=(15,10))
ax.plot(results['rtpr'],results['rfpr'])
ax.plot(results['stpr'],results['sfpr'])
ax.plot(results['afpr'],results['atpr'])
ax.plot(results['xtpr'],results['xfpr'])
ax.plot(results['ltpr'],results['lfpr'])

plt.legend(['Random forest :'+str(int(results['rrocscore']*100))+'%','svm :'+str(int(results['srocscore']*100))+'%','auto_encoders :'+str(int(results['arocscore']*100))+'%','xgb :'+str(int(results['xrocscore']*100))+'%','LSTM :'+str(int(results['lrocscore']*100))],fontsize=15,loc='lower right')
plt.title('ROC CURVES COMPARISON - KDD TRAIN+ TEST+ (65:35)',fontsize=20)
ax.plot([[0,0],[1,1]],'r--')
plt.ylabel('TRUE POSITIVE RATE',fontsize=15)
plt.xlabel('FALSE POSITIVE RATE',fontsize=15)

In [ ]:
fig,ax=plt.subplots(figsize=(15,10))
ax.plot(results['rrscre'],results['rpscre'])
ax.plot(results['srscre'],results['spscre'])
ax.plot(results['arscre'],results['apscre'])
ax.plot(results['xrscre'],results['xpscre'])
ax.plot(results['lrscre'],results['lpscre'])
plt.legend(['Random_forest :'+str(int(results['rprcscore']*100))+'%','svm :'+str(int(results['sprcscore']*100))+'%','auto_encoders :'+str(int(results['aprcscore']*100))+'%','xgb :'+str(int(results['xprcscore']*100))+'%','LSTM :'+str(int(results['lprcscore']*100))+'%'],fontsize=15)

plt.title('PRECISSION -RECALL CURVES COMPARISON COMBINED(65:35)',fontsize=20)
ax.plot([[0,0],[1,1]],'r--')
plt.ylabel('RECALL',fontsize=15)
plt.xlabel('PRECISION',fontsize=15)

In [ ]:
# Roc 
fig,ax=plt.subplots(figsize=(15,10))
x=int(results['rtime'])
ax.text(0,20+x,x, fontsize=12)
y=int(abs(results['xtime']))
ax.text(2,20+y,y, fontsize=12)
z=int(abs(results['stime']))
ax.text(1,5+z,z, fontsize=12)
w=int(results['atime'])
ax.text(3,20+w,w, fontsize=12)
v=int(results['ltime'])
ax.text(4,20+v,v, fontsize=12)


labels=['Random Forest','SVM','XGB','AUTO_ENOCDERS','LSTM']
plt.title('TRAINING TIME COMPARISON COMBINED(65:35)',fontsize=20)
plt.ylabel('TRAINING TIME in sec',fontsize=15)
plt.xlabel('ALGORITHMS',fontsize=15)
ax.bar(labels,[x,z,y,w,v])


In [ ]:
results['smap']

In [ ]:
fig,ax=plt.subplots(figsize=(15,10))
x=int(results['rmap']*100)
ax.text(0,1+x,x, fontsize=12)
y=int(abs(results['xmap']*100))
ax.text(2,1+y,y, fontsize=12)
z=int(results['smap']*100)
ax.text(1,1+z,z, fontsize=12)
w=int(results['amap']*100)
ax.text(3,1+w,w, fontsize=12)
v=int(results['lmap']*100)
ax.text(4,1+v,v, fontsize=12)




labels=['Random Forest','SVM','XGB' ,'AUTO_ENOCDERS','LSTM']
plt.title('MEAN AVERAGE PRECISION SCORES COMPARISON COMBINED(65:35)(values in %) - KDD TRAIN+ TEST+ ',fontsize=20)
plt.ylabel('MEAN AVERAGE PRECISION',fontsize=15)
plt.xlabel('ALGORITHMS',fontsize=15)
ax.bar(labels,[x,z,y,w,v])

In [ ]:
accu=[results['rscore'],results['sscore'],results['xscore'],results['ascore'],results['lscore']]
precision=[results['rprec'],results['sprec'],results['xprec'],results['aprec'],results['lprec']]
recal=[results['rrec'],results['srec'],results['xrec'],results['arec'],results['arec']]
maps=[results['rmap'],results['smap'],results['xmap'],results['amap'],results['lmap']]
fscore=[results['rf'],results['sf'],results['xf'],results['af'],results['lf']]


In [ ]:
cd=list([accu,precision,recal,maps,fscore])
ds=np.array(cd)
df=pd.DataFrame(ds*100)
df.columns=['random_Forest','SVM' ,'XGB'  , 'Auto_Encoders','LSTM']
df.index=['accuracy','precision','recal','maps','fscore']
df

In [ ]:
df.to_csv('comb_table(65:35).csv')

In [ ]:
results['xmap']

In [ ]:
x_train.shape